<h1> ECE4179 - Semi-Supervised Learning Project</h1>
<h2>Data</h2>

We will be using a dataset that can be obtained directly from the torchvision package. There are 10 classes and we will be training a CNN for the image classification task. We have training, validation and test sets that are labelled with the class, and a large unlabeled set.

We will simulating a low training data scenario by only sampling a small percentage of the labelled data (10%) as training data. The remaining examples will be used as the validation set.

To get the labelled data, change the dataset_dir to something suitable for your machine, and execute the following (you will then probably want to wrap the dataset objects in a PyTorch DataLoader):

In [2]:
import torch
import torch.nn as nn
from torchvision.datasets import STL10 as STL10
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import Subset
from copy import deepcopy
from torch.optim import Adam
import torch.optim as optim
from torchvision import models
from sklearn.metrics import f1_score, classification_report
import torch.nn.functional as F
import csv
import os
import random
import math
import time

####### CHANGE TO APPROPRIATE DIRECTORY TO STORE DATASET
dataset_dir = "../data"
# For MonARCH
# dataset_dir = "/mnt/lustre/projects/ds19/SHARED"

# All images are 3x96x96
image_size = 96
# batch size
batch_size = 128
# number of classes in our case is 10
num_classes = 10

<h3>Print the GPU type and use device</h3>

In [4]:
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"Using GPU: {gpu_name}")
else:
    print("No GPU available, using CPU")

# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Using GPU: NVIDIA TITAN RTX


<h3>Print the number of processors in cpu</h3>

In [6]:
import multiprocessing
print(multiprocessing.cpu_count())

16


<h3>Create the appropriate transforms</h3>

In [8]:
#Perform random crops and mirroring for data augmentation
transform_train = transforms.Compose(
    [transforms.RandomCrop(image_size, padding=4),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_unlabelled = transforms.Compose(
    [transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#No random 
transform_test = transforms.Compose(
    [transforms.CenterCrop(image_size),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


<h3>Create training and validation split</h3>

In [10]:
#Load train and validation sets
trainval_set = STL10(dataset_dir, split='train', transform=transform_train, download=True)

#Use 10% of data for training - simulating low data scenario
num_train = int(len(trainval_set)*0.1)

#Split data into train/val sets
torch.manual_seed(0) #Set torch's random seed so that random split of data is reproducible
train_set, val_set = random_split(trainval_set, [num_train, len(trainval_set)-num_train])

#Load test set
test_set = STL10(dataset_dir, split='test', transform=transform_test, download=False)

Files already downloaded and verified


<h3>Get the unlabelled data</h3>

In [12]:
unlabelled_set = STL10(dataset_dir, split='unlabeled', transform=transform_unlabelled, download=True)

Files already downloaded and verified


You may find later that you want to make changes to how the unlabelled data is loaded. This might require you sub-classing the STL10 class used above or to create your own dataloader similar to the Pytorch one.
https://pytorch.org/docs/stable/_modules/torchvision/datasets/stl10.html#STL10

<h3>Create the four dataloaders</h3>

In [15]:
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, num_workers=2)

valid_loader = DataLoader(val_set, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=2)

<h3>Accuracy</h3>

In [17]:
# Define the test function
def test_model(model, test_loader):
    # Define the device inside the function
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)  # Move the model to the appropriate device
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")


<h3>Marco F1 Score</h3>

In [19]:
# Define the test function to calculate F1 score
def test_model_with_f1(model, test_loader):
    # Define the device inside the function
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)  # Move the model to the appropriate device
    model.eval()  # Set model to evaluation mode
    
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            # Collect all predictions and labels for F1-score calculation
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Calculate the Macro F1-score for each class
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    # Alternatively, you can get a detailed report for all classes
    report = classification_report(all_labels, all_preds, target_names=[f"Class {i}" for i in range(10)])
    
    print(f"Macro F1-score: {f1}")
    print("Classification Report:\n", report)

### Define the training function

In [21]:
def train_model_with_grid_search(
    model,
    train_loader,
    valid_loader,
    num_classes,
    learning_rates,
    num_epochs_list,
    log_filename='training_log.csv',
    model_name='resnet',
    batch_size=128,
    patience=5  # Early stopping patience
):
    # Ensure 'logs' directory exists for saving the best model
    if not os.path.exists('logs'):
        os.makedirs('logs')

    ### Transfer Learning with Grid Search ###

    print("Starting Transfer Learning with Grid Search...")
    # Define different layer unfreezing configurations based on model type
    if model_name == 'resnet':
        unfreeze_configs = {
            'fc': ['fc'],
            'fc+layer4': ['layer4', 'fc'],
            'fc+layer3+layer4': ['layer3', 'layer4', 'fc'],
        }
    elif model_name == 'efficientnet':
        unfreeze_configs = {
            'fc': ['classifier.1'],
            'fc+features8': ['features.8', 'classifier.1'],
            'fc+features7+features8': ['features.7', 'features.8', 'classifier.1'],
        }
    elif model_name == 'vit':
        unfreeze_configs = {
            'fc': ['heads.head'],
            'fc+encoder11': ['encoder.layers.encoder_layer_11', 'heads.head'],
            'fc+encoder10+encoder11': ['encoder.layers.encoder_layer_10', 'encoder.layers.encoder_layer_11', 'heads.head'],
        }
    else:
        raise ValueError("Model name not recognized.")

    best_f1 = 0.0
    best_config = ''
    best_model_state = None

    # Open log file for recording training progress
    with open(os.path.join('logs', log_filename), mode='w', newline='') as log_file:
        log_writer = csv.writer(log_file)
        log_writer.writerow(['Configuration', 'Learning Rate', 'Num Epochs', 'Epoch', 'Training Loss', 'Validation Macro F1', 'Epoch Time (s)'])

        for config_name, layers_to_unfreeze in unfreeze_configs.items():
            for lr in learning_rates:
                for num_epochs in num_epochs_list:
                    print(f"\nConfiguration: {config_name}, Learning Rate: {lr}, Num Epochs: {num_epochs}")
                    # Create a new model by copying the original model
                    finetune_model = deepcopy(model)

                    # Modify the final layer based on the model's layer names
                    if model_name == 'resnet':
                        feature_dim = finetune_model.fc.in_features
                        finetune_model.fc = nn.Linear(feature_dim, num_classes)
                    elif model_name == 'efficientnet':
                        feature_dim = finetune_model.classifier[1].in_features
                        finetune_model.classifier[1] = nn.Linear(feature_dim, num_classes)
                    elif model_name == 'vit':
                        feature_dim = finetune_model.heads.head.in_features
                        finetune_model.heads.head = nn.Linear(feature_dim, num_classes)
                    else:
                        raise ValueError("Model name not recognized.")

                    # Freeze all layers first
                    for param in finetune_model.parameters():
                        param.requires_grad = False

                    # Unfreeze specified layers
                    for name, param in finetune_model.named_parameters():
                        for layer_name in layers_to_unfreeze:
                            if name.startswith(layer_name):
                                param.requires_grad = True

                    finetune_model = finetune_model.to(device)

                    # Define loss function and optimizer for fine-tuning
                    criterion = nn.CrossEntropyLoss()
                    optimizer = optim.Adam(filter(lambda p: p.requires_grad, finetune_model.parameters()), lr=lr)

                    # Learning rate scheduler for fine-tuning
                    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=patience, verbose=True)

                    # Early stopping variables
                    best_f1_local = 0.0
                    epochs_no_improve = 0

                    # Fine-tuning training loop
                    for epoch in range(num_epochs):
                        start_time = time.time()  # Start time for the epoch
                        finetune_model.train()
                        running_loss = 0.0
                        for inputs, labels in train_loader:
                            inputs, labels = inputs.to(device), labels.to(device)
                            optimizer.zero_grad()
                            outputs = finetune_model(inputs)
                            loss = criterion(outputs, labels)
                            loss.backward()
                            optimizer.step()
                            running_loss += loss.item()
                        avg_loss = running_loss / len(train_loader)

                        # Validation
                        finetune_model.eval()
                        all_labels = []
                        all_preds = []
                        with torch.no_grad():
                            for inputs, labels in valid_loader:
                                inputs, labels = inputs.to(device), labels.to(device)
                                outputs = finetune_model(inputs)
                                _, predicted = torch.max(outputs, 1)
                                all_labels.extend(labels.cpu().numpy())
                                all_preds.extend(predicted.cpu().numpy())

                        # Calculate Macro F1 score
                        f1 = f1_score(all_labels, all_preds, average='macro')

                        # Step the scheduler
                        scheduler.step(f1)

                        # Calculate epoch time
                        epoch_time = time.time() - start_time  # Calculate epoch duration

                        # Log results
                        log_writer.writerow([config_name, lr, num_epochs, epoch + 1, avg_loss, f1, epoch_time])
                        print(f"Config: {config_name}, LR: {lr}, Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Validation Macro F1: {f1:.4f}, Time: {epoch_time:.2f}s")

                        # Early stopping check
                        if f1 > best_f1_local:
                            best_f1_local = f1
                            epochs_no_improve = 0
                            # Save the best model for this configuration
                            best_model_state_local = deepcopy(finetune_model.state_dict())
                        else:
                            epochs_no_improve += 1

                        if epochs_no_improve >= patience:
                            print(f"Early stopping at epoch {epoch+1}")
                            break

                    # Update best model if current config is better
                    if best_f1_local > best_f1:
                        best_f1 = best_f1_local
                        best_config = f"{config_name}, LR: {lr}, Num Epochs: {num_epochs}"
                        best_model_state = best_model_state_local

    print(f"\nBest Configuration: {best_config} with Macro F1 Score: {best_f1}")
    # Save the best model
    torch.save(best_model_state, os.path.join('logs', f"best_model_{model_name}.pth"))


## Network

Let's use a ResNet18 architecture for our CNN...

## ResNet18

In [24]:
# We will keep this for later
model0 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)


for name, param in model0.named_parameters():
    print(f"Name: {name}, Shape: {param.shape}")

Name: conv1.weight, Shape: torch.Size([64, 3, 7, 7])
Name: bn1.weight, Shape: torch.Size([64])
Name: bn1.bias, Shape: torch.Size([64])
Name: layer1.0.conv1.weight, Shape: torch.Size([64, 64, 3, 3])
Name: layer1.0.bn1.weight, Shape: torch.Size([64])
Name: layer1.0.bn1.bias, Shape: torch.Size([64])
Name: layer1.0.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Name: layer1.0.bn2.weight, Shape: torch.Size([64])
Name: layer1.0.bn2.bias, Shape: torch.Size([64])
Name: layer1.1.conv1.weight, Shape: torch.Size([64, 64, 3, 3])
Name: layer1.1.bn1.weight, Shape: torch.Size([64])
Name: layer1.1.bn1.bias, Shape: torch.Size([64])
Name: layer1.1.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Name: layer1.1.bn2.weight, Shape: torch.Size([64])
Name: layer1.1.bn2.bias, Shape: torch.Size([64])
Name: layer2.0.conv1.weight, Shape: torch.Size([128, 64, 3, 3])
Name: layer2.0.bn1.weight, Shape: torch.Size([128])
Name: layer2.0.bn1.bias, Shape: torch.Size([128])
Name: layer2.0.conv2.weight, Shape: torch.Size(

Using cache found in C:\Users\惟神君/.cache\torch\hub\pytorch_vision_v0.10.0
G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
# Example usage with ResNet18
model_resnet18 = deepcopy(model0)  # assuming model0 is a pretrained resnet18
model_resnet18 = model_resnet18.to(device)

# Define learning rates and number of epochs for grid search
resnet18_learning_rates = [1e-5, 1e-4, 1e-3, 1e-2]
resnet18_num_epochs_list = [10, 15, 20]
patience = 5  # Early stopping patience

# Call the updated training function without the autoencoder
train_model_with_grid_search(
    model=model_resnet18,
    train_loader=train_loader,
    valid_loader=valid_loader,
    num_classes=num_classes,
    learning_rates=resnet18_learning_rates,
    num_epochs_list=resnet18_num_epochs_list,
    log_filename='resnet18_training_log.csv',
    model_name='resnet',
    batch_size=batch_size,
    patience=patience
)

Starting Transfer Learning with Grid Search...

Configuration: fc, Learning Rate: 1e-05, Num Epochs: 10


G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc, LR: 1e-05, Epoch [1/10], Loss: 2.6335, Validation Macro F1: 0.0681, Time: 17.80s
Config: fc, LR: 1e-05, Epoch [2/10], Loss: 2.5774, Validation Macro F1: 0.0682, Time: 17.68s
Config: fc, LR: 1e-05, Epoch [3/10], Loss: 2.5863, Validation Macro F1: 0.0729, Time: 17.69s
Config: fc, LR: 1e-05, Epoch [4/10], Loss: 2.5707, Validation Macro F1: 0.0808, Time: 17.99s
Config: fc, LR: 1e-05, Epoch [5/10], Loss: 2.5604, Validation Macro F1: 0.0843, Time: 18.41s
Config: fc, LR: 1e-05, Epoch [6/10], Loss: 2.5484, Validation Macro F1: 0.0789, Time: 18.27s
Config: fc, LR: 1e-05, Epoch [7/10], Loss: 2.5188, Validation Macro F1: 0.0950, Time: 17.39s
Config: fc, LR: 1e-05, Epoch [8/10], Loss: 2.5458, Validation Macro F1: 0.0903, Time: 17.85s
Config: fc, LR: 1e-05, Epoch [9/10], Loss: 2.5528, Validation Macro F1: 0.0872, Time: 17.61s
Config: fc, LR: 1e-05, Epoch [10/10], Loss: 2.5405, Validation Macro F1: 0.0974, Time: 17.72s

Configuration: fc, Learning Rate: 1e-05, Num Epochs: 15


G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc, LR: 1e-05, Epoch [1/15], Loss: 2.5154, Validation Macro F1: 0.0939, Time: 17.67s
Config: fc, LR: 1e-05, Epoch [2/15], Loss: 2.5170, Validation Macro F1: 0.0998, Time: 17.46s
Config: fc, LR: 1e-05, Epoch [3/15], Loss: 2.4892, Validation Macro F1: 0.0998, Time: 17.56s
Config: fc, LR: 1e-05, Epoch [4/15], Loss: 2.4840, Validation Macro F1: 0.0997, Time: 17.53s
Config: fc, LR: 1e-05, Epoch [5/15], Loss: 2.5020, Validation Macro F1: 0.0998, Time: 17.55s
Config: fc, LR: 1e-05, Epoch [6/15], Loss: 2.4751, Validation Macro F1: 0.1056, Time: 17.59s
Config: fc, LR: 1e-05, Epoch [7/15], Loss: 2.4805, Validation Macro F1: 0.1066, Time: 17.58s
Config: fc, LR: 1e-05, Epoch [8/15], Loss: 2.4802, Validation Macro F1: 0.1058, Time: 17.55s
Config: fc, LR: 1e-05, Epoch [9/15], Loss: 2.4857, Validation Macro F1: 0.1086, Time: 17.31s
Config: fc, LR: 1e-05, Epoch [10/15], Loss: 2.4561, Validation Macro F1: 0.1188, Time: 17.83s
Config: fc, LR: 1e-05, Epoch [11/15], Loss: 2.4731, Validation Macro 

...

Config: fc+layer3+layer4, LR: 0.01, Epoch [1/10], Loss: 3.3914, Validation Macro F1: 0.0192, Time: 18.22s
Config: fc+layer3+layer4, LR: 0.01, Epoch [2/10], Loss: 1.6994, Validation Macro F1: 0.0490, Time: 17.79s
Config: fc+layer3+layer4, LR: 0.01, Epoch [3/10], Loss: 1.7090, Validation Macro F1: 0.0187, Time: 18.28s
Config: fc+layer3+layer4, LR: 0.01, Epoch [4/10], Loss: 1.3040, Validation Macro F1: 0.0564, Time: 18.17s
Config: fc+layer3+layer4, LR: 0.01, Epoch [5/10], Loss: 1.1345, Validation Macro F1: 0.1204, Time: 18.10s
Config: fc+layer3+layer4, LR: 0.01, Epoch [6/10], Loss: 0.8543, Validation Macro F1: 0.2579, Time: 17.77s
Config: fc+layer3+layer4, LR: 0.01, Epoch [7/10], Loss: 0.7731, Validation Macro F1: 0.3588, Time: 17.62s
Config: fc+layer3+layer4, LR: 0.01, Epoch [8/10], Loss: 0.6974, Validation Macro F1: 0.4856, Time: 17.46s
Config: fc+layer3+layer4, LR: 0.01, Epoch [9/10], Loss: 0.5885, Validation Macro F1: 0.4364, Time: 17.94s
Config: fc+layer3+layer4, LR: 0.01, Epoch [10/

G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc+layer3+layer4, LR: 0.01, Epoch [1/15], Loss: 3.7285, Validation Macro F1: 0.0180, Time: 18.17s
Config: fc+layer3+layer4, LR: 0.01, Epoch [2/15], Loss: 2.9055, Validation Macro F1: 0.0693, Time: 18.78s
Config: fc+layer3+layer4, LR: 0.01, Epoch [3/15], Loss: 1.7528, Validation Macro F1: 0.0384, Time: 18.44s
Config: fc+layer3+layer4, LR: 0.01, Epoch [4/15], Loss: 1.3442, Validation Macro F1: 0.0911, Time: 18.18s
Config: fc+layer3+layer4, LR: 0.01, Epoch [5/15], Loss: 1.1584, Validation Macro F1: 0.1787, Time: 18.60s
Config: fc+layer3+layer4, LR: 0.01, Epoch [6/15], Loss: 1.0768, Validation Macro F1: 0.2648, Time: 18.41s
Config: fc+layer3+layer4, LR: 0.01, Epoch [7/15], Loss: 1.0250, Validation Macro F1: 0.3269, Time: 18.34s
Config: fc+layer3+layer4, LR: 0.01, Epoch [8/15], Loss: 0.8181, Validation Macro F1: 0.3522, Time: 18.51s
Config: fc+layer3+layer4, LR: 0.01, Epoch [9/15], Loss: 0.7420, Validation Macro F1: 0.4126, Time: 18.38s
Config: fc+layer3+layer4, LR: 0.01, Epoch [10/

G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc+layer3+layer4, LR: 0.01, Epoch [1/20], Loss: 3.3549, Validation Macro F1: 0.0450, Time: 18.41s
Config: fc+layer3+layer4, LR: 0.01, Epoch [2/20], Loss: 1.8253, Validation Macro F1: 0.0854, Time: 18.32s
Config: fc+layer3+layer4, LR: 0.01, Epoch [3/20], Loss: 1.4934, Validation Macro F1: 0.0222, Time: 18.30s
Config: fc+layer3+layer4, LR: 0.01, Epoch [4/20], Loss: 1.1266, Validation Macro F1: 0.0441, Time: 18.37s
Config: fc+layer3+layer4, LR: 0.01, Epoch [5/20], Loss: 0.8804, Validation Macro F1: 0.1827, Time: 18.05s
Config: fc+layer3+layer4, LR: 0.01, Epoch [6/20], Loss: 0.7580, Validation Macro F1: 0.2843, Time: 18.02s
Config: fc+layer3+layer4, LR: 0.01, Epoch [7/20], Loss: 0.7024, Validation Macro F1: 0.3294, Time: 18.06s
Config: fc+layer3+layer4, LR: 0.01, Epoch [8/20], Loss: 0.5386, Validation Macro F1: 0.3689, Time: 18.15s
Config: fc+layer3+layer4, LR: 0.01, Epoch [9/20], Loss: 0.4896, Validation Macro F1: 0.5679, Time: 18.27s
Config: fc+layer3+layer4, LR: 0.01, Epoch [10/

In [26]:
# Initialize the ResNet18 model with pretrained weights
best_model_resnet = models.resnet18(pretrained=True)

# Replace the final fully connected layer to match the number of classes
best_model_resnet.fc = nn.Linear(best_model_resnet.fc.in_features, num_classes)

# Load the best model weights (ensure map_location if needed)
best_model_resnet.load_state_dict(torch.load('logs/best_model_resnet.pth', map_location=device))

# Move the model to the appropriate device (CPU or GPU)
best_model_resnet = best_model_resnet.to(device)

# Set the model to evaluation mode
best_model_resnet.eval()

G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\惟神君\AppData\Local\Temp\ipykernel_16392\2960042565.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#un

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [27]:
# Call the test functions
test_model(best_model_resnet, test_loader)

Test Accuracy: 79.9%


In [28]:
test_model_with_f1(best_model_resnet, test_loader)

Macro F1-score: 0.7972120266791742
Classification Report:
               precision    recall  f1-score   support

     Class 0       0.82      0.90      0.86       800
     Class 1       0.82      0.85      0.84       800
     Class 2       0.90      0.92      0.91       800
     Class 3       0.73      0.65      0.69       800
     Class 4       0.75      0.79      0.77       800
     Class 5       0.71      0.62      0.66       800
     Class 6       0.77      0.82      0.80       800
     Class 7       0.75      0.81      0.78       800
     Class 8       0.88      0.88      0.88       800
     Class 9       0.85      0.75      0.80       800

    accuracy                           0.80      8000
   macro avg       0.80      0.80      0.80      8000
weighted avg       0.80      0.80      0.80      8000



## EfficientNet

In [30]:
# Load pretrained EfficientNet-B0 model from torchvision hub
model1 = torch.hub.load('pytorch/vision', 'efficientnet_b0', weights="EfficientNet_B0_Weights.IMAGENET1K_V1")

for name, param in model1.named_parameters():
    print(f"Name: {name}, Shape: {param.shape}")

Name: features.0.0.weight, Shape: torch.Size([32, 3, 3, 3])
Name: features.0.1.weight, Shape: torch.Size([32])
Name: features.0.1.bias, Shape: torch.Size([32])
Name: features.1.0.block.0.0.weight, Shape: torch.Size([32, 1, 3, 3])
Name: features.1.0.block.0.1.weight, Shape: torch.Size([32])
Name: features.1.0.block.0.1.bias, Shape: torch.Size([32])
Name: features.1.0.block.1.fc1.weight, Shape: torch.Size([8, 32, 1, 1])
Name: features.1.0.block.1.fc1.bias, Shape: torch.Size([8])
Name: features.1.0.block.1.fc2.weight, Shape: torch.Size([32, 8, 1, 1])
Name: features.1.0.block.1.fc2.bias, Shape: torch.Size([32])
Name: features.1.0.block.2.0.weight, Shape: torch.Size([16, 32, 1, 1])
Name: features.1.0.block.2.1.weight, Shape: torch.Size([16])
Name: features.1.0.block.2.1.bias, Shape: torch.Size([16])
Name: features.2.0.block.0.0.weight, Shape: torch.Size([96, 16, 1, 1])
Name: features.2.0.block.0.1.weight, Shape: torch.Size([96])
Name: features.2.0.block.0.1.bias, Shape: torch.Size([96])
Nam

Using cache found in C:\Users\惟神君/.cache\torch\hub\pytorch_vision_main


In [31]:
# Example usage with EfficientNetB0
model_efficientnetb0 = deepcopy(model1)  # assuming model1 is a pretrained efficientnetb0
model_efficientnetb0 = model_efficientnetb0.to(device)

# Define learning rates and number of epochs for grid search
efficientnet_learning_rates = [1e-5, 1e-4, 1e-3, 1e-2]
efficientnet_num_epochs_list = [10, 15, 20]
patience = 5  # Early stopping patience

# Call the updated training function without the autoencoder
train_model_with_grid_search(
    model=model_efficientnetb0,
    train_loader=train_loader,
    valid_loader=valid_loader,
    num_classes=num_classes,
    learning_rates=efficientnet_learning_rates,
    num_epochs_list=efficientnet_num_epochs_list,
    log_filename='efficientnetb0_training_log.csv',
    model_name='efficientnet',
    batch_size=batch_size,
    patience=patience
)

Starting Transfer Learning with Grid Search...

Configuration: fc, Learning Rate: 1e-05, Num Epochs: 10


G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc, LR: 1e-05, Epoch [1/10], Loss: 2.3544, Validation Macro F1: 0.0957, Time: 18.44s
Config: fc, LR: 1e-05, Epoch [2/10], Loss: 2.3548, Validation Macro F1: 0.0916, Time: 18.35s
Config: fc, LR: 1e-05, Epoch [3/10], Loss: 2.3615, Validation Macro F1: 0.1003, Time: 18.28s
Config: fc, LR: 1e-05, Epoch [4/10], Loss: 2.3553, Validation Macro F1: 0.0978, Time: 18.36s
Config: fc, LR: 1e-05, Epoch [5/10], Loss: 2.3451, Validation Macro F1: 0.1038, Time: 18.54s
Config: fc, LR: 1e-05, Epoch [6/10], Loss: 2.3460, Validation Macro F1: 0.1019, Time: 18.23s
Config: fc, LR: 1e-05, Epoch [7/10], Loss: 2.3471, Validation Macro F1: 0.1001, Time: 18.16s
Config: fc, LR: 1e-05, Epoch [8/10], Loss: 2.3340, Validation Macro F1: 0.1058, Time: 18.25s
Config: fc, LR: 1e-05, Epoch [9/10], Loss: 2.3192, Validation Macro F1: 0.0982, Time: 18.39s
Config: fc, LR: 1e-05, Epoch [10/10], Loss: 2.3469, Validation Macro F1: 0.1105, Time: 18.33s

Configuration: fc, Learning Rate: 1e-05, Num Epochs: 15


G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc, LR: 1e-05, Epoch [1/15], Loss: 2.3488, Validation Macro F1: 0.1115, Time: 18.29s
Config: fc, LR: 1e-05, Epoch [2/15], Loss: 2.3388, Validation Macro F1: 0.1117, Time: 18.35s
Config: fc, LR: 1e-05, Epoch [3/15], Loss: 2.3523, Validation Macro F1: 0.1119, Time: 18.34s
Config: fc, LR: 1e-05, Epoch [4/15], Loss: 2.3341, Validation Macro F1: 0.1146, Time: 18.31s
Config: fc, LR: 1e-05, Epoch [5/15], Loss: 2.3376, Validation Macro F1: 0.1163, Time: 18.23s
Config: fc, LR: 1e-05, Epoch [6/15], Loss: 2.3535, Validation Macro F1: 0.1183, Time: 18.54s
Config: fc, LR: 1e-05, Epoch [7/15], Loss: 2.3349, Validation Macro F1: 0.1205, Time: 18.31s
Config: fc, LR: 1e-05, Epoch [8/15], Loss: 2.3291, Validation Macro F1: 0.1152, Time: 18.08s
Config: fc, LR: 1e-05, Epoch [9/15], Loss: 2.3254, Validation Macro F1: 0.1229, Time: 18.04s
Config: fc, LR: 1e-05, Epoch [10/15], Loss: 2.3484, Validation Macro F1: 0.1179, Time: 18.12s
Config: fc, LR: 1e-05, Epoch [11/15], Loss: 2.3362, Validation Macro 

...

Config: fc+features7+features8, LR: 0.01, Epoch [1/10], Loss: 1.5721, Validation Macro F1: 0.6403, Time: 18.16s
Config: fc+features7+features8, LR: 0.01, Epoch [2/10], Loss: 0.6339, Validation Macro F1: 0.6442, Time: 18.33s
Config: fc+features7+features8, LR: 0.01, Epoch [3/10], Loss: 0.4206, Validation Macro F1: 0.6650, Time: 18.48s
Config: fc+features7+features8, LR: 0.01, Epoch [4/10], Loss: 0.2702, Validation Macro F1: 0.6127, Time: 18.19s
Config: fc+features7+features8, LR: 0.01, Epoch [5/10], Loss: 0.3071, Validation Macro F1: 0.6515, Time: 18.10s
Config: fc+features7+features8, LR: 0.01, Epoch [6/10], Loss: 0.2627, Validation Macro F1: 0.6511, Time: 18.27s
Config: fc+features7+features8, LR: 0.01, Epoch [7/10], Loss: 0.3489, Validation Macro F1: 0.6488, Time: 18.40s
Config: fc+features7+features8, LR: 0.01, Epoch [8/10], Loss: 0.1925, Validation Macro F1: 0.6394, Time: 17.99s
Early stopping at epoch 8

Configuration: fc+features7+features8, Learning Rate: 0.01, Num Epochs: 15


G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc+features7+features8, LR: 0.01, Epoch [1/15], Loss: 1.6760, Validation Macro F1: 0.5896, Time: 18.24s
Config: fc+features7+features8, LR: 0.01, Epoch [2/15], Loss: 0.7068, Validation Macro F1: 0.6688, Time: 18.17s
Config: fc+features7+features8, LR: 0.01, Epoch [3/15], Loss: 0.5205, Validation Macro F1: 0.6195, Time: 18.38s
Config: fc+features7+features8, LR: 0.01, Epoch [4/15], Loss: 0.2438, Validation Macro F1: 0.6705, Time: 17.70s
Config: fc+features7+features8, LR: 0.01, Epoch [5/15], Loss: 0.2335, Validation Macro F1: 0.6698, Time: 17.96s
Config: fc+features7+features8, LR: 0.01, Epoch [6/15], Loss: 0.1808, Validation Macro F1: 0.6548, Time: 17.60s
Config: fc+features7+features8, LR: 0.01, Epoch [7/15], Loss: 0.1616, Validation Macro F1: 0.6549, Time: 17.59s
Config: fc+features7+features8, LR: 0.01, Epoch [8/15], Loss: 0.1977, Validation Macro F1: 0.6715, Time: 17.95s
Config: fc+features7+features8, LR: 0.01, Epoch [9/15], Loss: 0.1160, Validation Macro F1: 0.6442, Time:

G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc+features7+features8, LR: 0.01, Epoch [1/20], Loss: 1.6623, Validation Macro F1: 0.5834, Time: 18.31s
Config: fc+features7+features8, LR: 0.01, Epoch [2/20], Loss: 0.6127, Validation Macro F1: 0.6122, Time: 18.61s
Config: fc+features7+features8, LR: 0.01, Epoch [3/20], Loss: 0.4724, Validation Macro F1: 0.6628, Time: 18.06s
Config: fc+features7+features8, LR: 0.01, Epoch [4/20], Loss: 0.3642, Validation Macro F1: 0.6462, Time: 18.33s
Config: fc+features7+features8, LR: 0.01, Epoch [5/20], Loss: 0.3088, Validation Macro F1: 0.6576, Time: 18.15s
Config: fc+features7+features8, LR: 0.01, Epoch [6/20], Loss: 0.2483, Validation Macro F1: 0.6498, Time: 18.34s
Config: fc+features7+features8, LR: 0.01, Epoch [7/20], Loss: 0.2530, Validation Macro F1: 0.6505, Time: 18.24s
Config: fc+features7+features8, LR: 0.01, Epoch [8/20], Loss: 0.2635, Validation Macro F1: 0.6698, Time: 18.26s
Config: fc+features7+features8, LR: 0.01, Epoch [9/20], Loss: 0.1845, Validation Macro F1: 0.6551, Time:

In [32]:
# Initialize the EfficientNetB0 model with pretrained weights
best_model_efficientnet = models.efficientnet_b0(pretrained=True)

# Replace the final classifier layer to match the number of classes
best_model_efficientnet.classifier[1] = nn.Linear(best_model_efficientnet.classifier[1].in_features, num_classes)

# Load the best model weights onto the appropriate device
best_model_efficientnet.load_state_dict(torch.load('logs/best_model_efficientnet.pth', map_location=device))

# Move the model to the device (if not already on the correct device)
best_model_efficientnet = best_model_efficientnet.to(device)

# Set the model to evaluation mode
best_model_efficientnet.eval()

G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\惟神君\AppData\Local\Temp\ipykernel_16392\3387589344.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [33]:
# Call the test function
test_model(best_model_efficientnet, test_loader)

Test Accuracy: 75.925%


In [34]:
# Call the function to calculate and print F1-scores
test_model_with_f1(best_model_efficientnet, test_loader)

Macro F1-score: 0.7584632465836025
Classification Report:
               precision    recall  f1-score   support

     Class 0       0.82      0.69      0.75       800
     Class 1       0.86      0.71      0.78       800
     Class 2       0.83      0.92      0.88       800
     Class 3       0.63      0.69      0.66       800
     Class 4       0.68      0.77      0.72       800
     Class 5       0.78      0.61      0.68       800
     Class 6       0.70      0.85      0.77       800
     Class 7       0.81      0.75      0.78       800
     Class 8       0.81      0.83      0.82       800
     Class 9       0.74      0.77      0.75       800

    accuracy                           0.76      8000
   macro avg       0.77      0.76      0.76      8000
weighted avg       0.77      0.76      0.76      8000



## Vision Transformer (ViT)

In [36]:
# Set image size to 224x224 to match the input size of ViT
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomCrop(224, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_unlabelled = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [37]:
# Load train and validation sets without redownloading data
trainval_set = STL10(dataset_dir, split='train', transform=transform_train, download=False)

# Use 10% of the data for training (simulating a low data scenario)
num_train = int(len(trainval_set) * 0.1)

# Split data into train/validation sets with a fixed random seed
torch.manual_seed(0)  # Ensure reproducibility
train_set, val_set = random_split(trainval_set, [num_train, len(trainval_set) - num_train])

# Load test set without redownloading data
test_set = STL10(dataset_dir, split='test', transform=transform_test, download=False)

In [38]:
unlabelled_set = STL10(dataset_dir, split='unlabeled', transform=transform_unlabelled, download=False)

# Determine the size of the subset (1/1000 of the full dataset)
subset_size = len(unlabelled_set) // 1000  # This will be 100 samples

# Randomly select indices for the subset
random_indices = random.sample(range(len(unlabelled_set)), subset_size)

# Create a subset of the unlabelled dataset
unlabelled_subset = Subset(unlabelled_set, random_indices)

# Now, create the DataLoader using the subset
unlabelled_loader = DataLoader(unlabelled_subset, shuffle=True, batch_size=batch_size, num_workers=2)

In [39]:
# Create DataLoader for train, validation, and test sets
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, num_workers=2)

valid_loader = DataLoader(val_set, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=2)

In [40]:
# Load pretrained Vision Transformer (ViT) model from torchvision models
model2 = models.vit_b_16(pretrained=True)

# Print the model structure to verify the changes
for name, param in model2.named_parameters():
    print(f"Name: {name}, Shape: {param.shape}")

G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Name: class_token, Shape: torch.Size([1, 1, 768])
Name: conv_proj.weight, Shape: torch.Size([768, 3, 16, 16])
Name: conv_proj.bias, Shape: torch.Size([768])
Name: encoder.pos_embedding, Shape: torch.Size([1, 197, 768])
Name: encoder.layers.encoder_layer_0.ln_1.weight, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.ln_1.bias, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.self_attention.in_proj_weight, Shape: torch.Size([2304, 768])
Name: encoder.layers.encoder_layer_0.self_attention.in_proj_bias, Shape: torch.Size([2304])
Name: encoder.layers.encoder_layer_0.self_attention.out_proj.weight, Shape: torch.Size([768, 768])
Name: encoder.layers.encoder_layer_0.self_attention.out_proj.bias, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.ln_2.weight, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.ln_2.bias, Shape: torch.Size([768])
Name: encoder.layers.encoder_layer_0.mlp.0.weight, Shape: torch.Size([3072, 768])
Name: encoder.layers.

In [41]:
# Example usage with Vision Transformer (ViT)
model_vit = deepcopy(model2)  # assuming model2 is a pretrained Vision Transformer (ViT)
model_vit = model_vit.to(device)

# Define learning rates and number of epochs for grid search
vit_learning_rates = [1e-5, 1e-4, 1e-3, 1e-2]
vit_num_epochs_list = [10, 15, 20]
patience = 5  # Early stopping patience

# Call the updated training function without the autoencoder
train_model_with_grid_search(
    model=model_vit,
    train_loader=train_loader,
    valid_loader=valid_loader,
    num_classes=num_classes,
    learning_rates=vit_learning_rates,
    num_epochs_list=vit_num_epochs_list,
    log_filename='vit_training_log.csv',
    model_name='vit',
    batch_size=batch_size,
    patience=patience
)

Starting Transfer Learning with Grid Search...

Configuration: fc, Learning Rate: 1e-05, Num Epochs: 10


G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
G:\Anaconda3\anaconda\lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Config: fc, LR: 1e-05, Epoch [1/10], Loss: 2.3624, Validation Macro F1: 0.1000, Time: 39.54s
Config: fc, LR: 1e-05, Epoch [2/10], Loss: 2.3526, Validation Macro F1: 0.1069, Time: 36.82s
Config: fc, LR: 1e-05, Epoch [3/10], Loss: 2.3451, Validation Macro F1: 0.1051, Time: 35.68s
Config: fc, LR: 1e-05, Epoch [4/10], Loss: 2.3372, Validation Macro F1: 0.1114, Time: 36.29s
Config: fc, LR: 1e-05, Epoch [5/10], Loss: 2.3298, Validation Macro F1: 0.1122, Time: 35.80s
Config: fc, LR: 1e-05, Epoch [6/10], Loss: 2.3345, Validation Macro F1: 0.1169, Time: 37.39s
Config: fc, LR: 1e-05, Epoch [7/10], Loss: 2.3244, Validation Macro F1: 0.1180, Time: 38.21s
Config: fc, LR: 1e-05, Epoch [8/10], Loss: 2.3157, Validation Macro F1: 0.1231, Time: 37.75s
Config: fc, LR: 1e-05, Epoch [9/10], Loss: 2.3051, Validation Macro F1: 0.1264, Time: 43.02s
Config: fc, LR: 1e-05, Epoch [10/10], Loss: 2.3062, Validation Macro F1: 0.1316, Time: 36.66s

Configuration: fc, Learning Rate: 1e-05, Num Epochs: 15


G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc, LR: 1e-05, Epoch [1/15], Loss: 2.3326, Validation Macro F1: 0.0975, Time: 41.18s
Config: fc, LR: 1e-05, Epoch [2/15], Loss: 2.3324, Validation Macro F1: 0.1027, Time: 39.70s
Config: fc, LR: 1e-05, Epoch [3/15], Loss: 2.3258, Validation Macro F1: 0.1037, Time: 41.45s
Config: fc, LR: 1e-05, Epoch [4/15], Loss: 2.3180, Validation Macro F1: 0.1032, Time: 36.82s
Config: fc, LR: 1e-05, Epoch [5/15], Loss: 2.3075, Validation Macro F1: 0.1094, Time: 42.32s
Config: fc, LR: 1e-05, Epoch [6/15], Loss: 2.3011, Validation Macro F1: 0.1117, Time: 37.51s
Config: fc, LR: 1e-05, Epoch [7/15], Loss: 2.2995, Validation Macro F1: 0.1188, Time: 35.60s
Config: fc, LR: 1e-05, Epoch [8/15], Loss: 2.2935, Validation Macro F1: 0.1227, Time: 39.96s
Config: fc, LR: 1e-05, Epoch [9/15], Loss: 2.2898, Validation Macro F1: 0.1236, Time: 34.96s
Config: fc, LR: 1e-05, Epoch [10/15], Loss: 2.2821, Validation Macro F1: 0.1286, Time: 35.06s
Config: fc, LR: 1e-05, Epoch [11/15], Loss: 2.2752, Validation Macro 

...

Config: fc+encoder10+encoder11, LR: 0.01, Epoch [1/10], Loss: 3.2879, Validation Macro F1: 0.2555, Time: 34.28s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [2/10], Loss: 3.3281, Validation Macro F1: 0.0692, Time: 34.09s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [3/10], Loss: 2.0931, Validation Macro F1: 0.4642, Time: 34.08s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [4/10], Loss: 1.0540, Validation Macro F1: 0.6102, Time: 34.09s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [5/10], Loss: 0.5927, Validation Macro F1: 0.7960, Time: 34.09s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [6/10], Loss: 0.2406, Validation Macro F1: 0.9240, Time: 34.32s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [7/10], Loss: 0.1116, Validation Macro F1: 0.9320, Time: 34.28s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [8/10], Loss: 0.0670, Validation Macro F1: 0.9408, Time: 34.08s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [9/10], Loss: 0.0244, Validation Macro F1: 0.9473, Time:

G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc+encoder10+encoder11, LR: 0.01, Epoch [1/15], Loss: 3.3979, Validation Macro F1: 0.0794, Time: 34.24s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [2/15], Loss: 3.0934, Validation Macro F1: 0.2328, Time: 34.10s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [3/15], Loss: 1.8679, Validation Macro F1: 0.4414, Time: 34.12s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [4/15], Loss: 1.1840, Validation Macro F1: 0.5177, Time: 34.30s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [5/15], Loss: 0.8099, Validation Macro F1: 0.6696, Time: 34.23s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [6/15], Loss: 0.5926, Validation Macro F1: 0.7122, Time: 34.29s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [7/15], Loss: 0.3805, Validation Macro F1: 0.8526, Time: 34.30s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [8/15], Loss: 0.2385, Validation Macro F1: 0.9103, Time: 34.12s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [9/15], Loss: 0.1350, Validation Macro F1: 0.9208, Time:

G:\Anaconda3\anaconda\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: fc+encoder10+encoder11, LR: 0.01, Epoch [1/20], Loss: 3.7331, Validation Macro F1: 0.1381, Time: 34.28s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [2/20], Loss: 4.3614, Validation Macro F1: 0.0884, Time: 34.32s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [3/20], Loss: 3.3803, Validation Macro F1: 0.1980, Time: 34.52s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [4/20], Loss: 1.8632, Validation Macro F1: 0.3098, Time: 34.41s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [5/20], Loss: 1.3158, Validation Macro F1: 0.5601, Time: 34.31s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [6/20], Loss: 0.8540, Validation Macro F1: 0.6637, Time: 34.30s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [7/20], Loss: 0.5303, Validation Macro F1: 0.7390, Time: 34.31s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [8/20], Loss: 0.3184, Validation Macro F1: 0.8722, Time: 34.25s
Config: fc+encoder10+encoder11, LR: 0.01, Epoch [9/20], Loss: 0.1959, Validation Macro F1: 0.9215, Time:

In [42]:
# Initialize the ViT model with pretrained weights
best_model_vit = models.vit_b_16(pretrained=True)

# Replace the final head to match the number of classes
best_model_vit.heads.head = nn.Linear(best_model_vit.heads.head.in_features, num_classes)

# Load the best model weights onto the appropriate device
best_model_vit.load_state_dict(torch.load('logs/best_model_vit.pth', map_location=device))

# Move the model to the device (if not already on the correct device)
best_model_vit = best_model_vit.to(device)

# Set the model to evaluation mode
best_model_vit.eval()

G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
G:\Anaconda3\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\惟神君\AppData\Local\Temp\ipykernel_16392\432782011.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#unt

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [43]:
# Call the test function
test_model(best_model_vit, test_loader)

Test Accuracy: 97.9125%


In [44]:
# Call the function to calculate and print F1-scores
test_model_with_f1(best_model_vit, test_loader)

Macro F1-score: 0.9791382683901115
Classification Report:
               precision    recall  f1-score   support

     Class 0       0.98      0.99      0.99       800
     Class 1       0.99      0.99      0.99       800
     Class 2       1.00      0.97      0.98       800
     Class 3       0.98      0.96      0.97       800
     Class 4       0.97      0.95      0.96       800
     Class 5       0.96      0.98      0.97       800
     Class 6       0.95      0.98      0.96       800
     Class 7       0.99      0.99      0.99       800
     Class 8       0.99      0.98      0.99       800
     Class 9       0.97      1.00      0.98       800

    accuracy                           0.98      8000
   macro avg       0.98      0.98      0.98      8000
weighted avg       0.98      0.98      0.98      8000

